In [1]:
# AFTER restarting Jupyter kernel or camera service
# Test just ONE camera at a time

from jetracer.camera_utils import JetRacerCamera

print("🔍 SINGLE CAMERA TEST AFTER RESET")
print("=" * 40)

try:
    # Test just safe mode
    camera = JetRacerCamera('safe')
    
    if camera.start():
        img = camera.read()
        if img is not None:
            print(f"✅ Single camera test SUCCESS: {img.shape}")
            print(f"✅ Mode: {camera.mode}")
        camera.stop()
        print("✅ Camera stopped cleanly")
    else:
        print("❌ Camera failed to start")
        
except Exception as e:
    print(f"❌ Single camera test failed: {e}")

🔍 SINGLE CAMERA TEST AFTER RESET
GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3280 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3280 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: Running with following settings:
   Camera index = 0 
   Camera mode  = 4

[ WARN:0@0.627] global cap_gstreamer.cpp:1728 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


✓ Camera started successfully
✓ Raw image shape: (480, 640, 3)
✓ Processed image shape: (480, 640, 3)
✅ Single camera test SUCCESS: (480, 640, 3)
✅ Mode: safe
✓ JetRacer camera stopped
✅ Camera stopped cleanly


In [4]:
# Test your updated camera utilities
# Run this in a Jupyter notebook AFTER reinstalling the package

print("=== TESTING UPDATED JETRACER CAMERA UTILS ===")

# Make sure to restart kernel or clear any existing camera variables
try:
    if 'camera' in globals():
        camera.running = False
        del camera
    import time
    time.sleep(2)
except:
    pass

# Test import
try:
    from jetracer.camera_utils import JetRacerCamera
    print("✓ Successfully imported updated JetRacerCamera")
except ImportError as e:
    print(f"✗ Import failed: {e}")
    print("Make sure you reinstalled the package!")
    exit()

# Test each mode
test_modes = ['safe', 'inference', 'training']

for mode in test_modes:
    print(f"\n{'='*20} Testing {mode} mode {'='*20}")
    
    try:
        # Create camera
        camera = JetRacerCamera(mode)
        
        # Start camera
        if camera.start():
            print(f"✓ {mode} camera started successfully")
            
            # Test image capture
            img = camera.read()
            if img is not None:
                print(f"✓ Image captured: {img.shape}")
                print(f"✓ Image dtype: {img.dtype}")
                print(f"✓ Camera width: {camera.width}")
                print(f"✓ Camera height: {camera.height}")
                
                # Test .value property
                value_img = camera.value
                if value_img is not None:
                    print(f"✓ .value property: {value_img.shape}")
                
                # Test multiple captures for stability
                print("Testing stability...")
                stable_count = 0
                for i in range(5):
                    test_img = camera.read()
                    if test_img is not None:
                        stable_count += 1
                    time.sleep(0.2)
                
                print(f"✓ Stable captures: {stable_count}/5")
                
                if mode == 'safe':
                    # Test raw_value for safe mode
                    raw_img = camera.raw_value
                    if raw_img is not None:
                        print(f"✓ Raw image: {raw_img.shape}")
                
            else:
                print(f"✗ No image captured in {mode} mode")
            
            # Stop camera
            camera.stop()
            
        else:
            print(f"✗ {mode} camera failed to start")
            
    except Exception as e:
        print(f"✗ {mode} mode error: {e}")
        try:
            camera.stop()
        except:
            pass
    
    # Clean up and wait between tests
    try:
        del camera
    except:
        pass
    time.sleep(2)

print("\n🎉 CAMERA UTILS TESTING COMPLETE! 🎉")

print("\n" + "="*50)
print("YOUR JETRACER CAMERA IS NOW READY TO USE!")
print("="*50)
print("\nUsage examples:")
print("# For road following (AI models):")
print("from jetracer.camera_utils import JetRacerCamera")
print("camera = JetRacerCamera('inference')")
print("camera.start()")
print("image = camera.read()  # Gets 224x224 image")
print("")
print("# For data collection:")
print("camera = JetRacerCamera('training')")
print("camera.start()")
print("")
print("# For testing/debugging:")
print("camera = JetRacerCamera('safe')")
print("camera.start()")
print("image = camera.read()  # Gets 640x480 image")
print("="*50)

=== TESTING UPDATED JETRACER CAMERA UTILS ===
✓ Successfully imported updated JetRacerCamera

==================== Testing safe mode ====================
✗ safe mode error: Could not initialize camera.  Please see error trace.


Error generated. /dvs/git/dirty/git-master_linux/multimedia/nvgstreamer/gst-nvarguscamera/gstnvarguscamerasrc.cpp, execute:805 Failed to create CaptureSession
[ WARN:0@281.682] global cap_gstreamer.cpp:1728 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1



==================== Testing inference mode ====================
✗ inference mode error: Could not initialize camera.  Please see error trace.


Error generated. /dvs/git/dirty/git-master_linux/multimedia/nvgstreamer/gst-nvarguscamera/gstnvarguscamerasrc.cpp, execute:805 Failed to create CaptureSession
[ WARN:0@283.706] global cap_gstreamer.cpp:1728 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1



==================== Testing training mode ====================
✗ training mode error: Could not initialize camera.  Please see error trace.


Error generated. /dvs/git/dirty/git-master_linux/multimedia/nvgstreamer/gst-nvarguscamera/gstnvarguscamerasrc.cpp, execute:805 Failed to create CaptureSession
[ WARN:0@285.739] global cap_gstreamer.cpp:1728 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1



🎉 CAMERA UTILS TESTING COMPLETE! 🎉

YOUR JETRACER CAMERA IS NOW READY TO USE!

Usage examples:
# For road following (AI models):
from jetracer.camera_utils import JetRacerCamera
camera = JetRacerCamera('inference')
camera.start()
image = camera.read()  # Gets 224x224 image

# For data collection:
camera = JetRacerCamera('training')
camera.start()

# For testing/debugging:
camera = JetRacerCamera('safe')
camera.start()
image = camera.read()  # Gets 640x480 image


In [3]:
# Minimal camera test after service reset
# Run this AFTER running the terminal commands above

import time

print("=== MINIMAL CAMERA TEST ===")

# Wait for service to fully restart
print("Waiting for camera service to stabilize...")
time.sleep(5)

# Test 1: Check if camera device exists
import os
video_devices = [f for f in os.listdir('/dev') if f.startswith('video')]
print(f"Video devices found: {video_devices}")

if not video_devices:
    print("✗ No video devices found! Camera hardware issue.")
    exit()

# Test 2: Try direct gstreamer pipeline (most basic test)
print("Testing direct GStreamer access...")
try:
    import subprocess
    
    # Test if gstreamer can access the camera
    cmd = [
        'gst-launch-1.0', 
        'nvarguscamerasrc', 
        'num-buffers=1', 
        '!', 'fakesink'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print("✓ GStreamer can access camera directly")
    else:
        print(f"✗ GStreamer error: {result.stderr}")
        
except Exception as e:
    print(f"GStreamer test failed: {e}")

# Test 3: Try OpenCV with different backends
print("Testing OpenCV camera access...")

import cv2

# Try different camera indices and backends
test_configs = [
    (0, cv2.CAP_GSTREAMER),
    (0, cv2.CAP_V4L2),
    (1, cv2.CAP_GSTREAMER),
]

for cam_index, backend in test_configs:
    try:
        print(f"Trying camera {cam_index} with backend {backend}...")
        
        cap = cv2.VideoCapture(cam_index, backend)
        
        if cap.isOpened():
            print(f"✓ Camera {cam_index} opened with backend {backend}")
            
            # Try to read a frame
            ret, frame = cap.read()
            if ret and frame is not None:
                print(f"✓ Frame captured: {frame.shape}")
                cap.release()
                
                # If this works, try with jetcam
                print("Testing with jetcam...")
                from jetcam.csi_camera import CSICamera
                
                # Use very basic settings
                camera = CSICamera(width=640, height=480, capture_fps=21)
                camera.running = True
                time.sleep(3)
                
                img = camera.value
                if img is not None:
                    print(f"✓ JetCam SUCCESS: {img.shape}")
                    camera.running = False
                    print("🎉 CAMERA IS WORKING! 🎉")
                    break
                else:
                    print("✗ JetCam failed to capture")
                    camera.running = False
            else:
                print(f"✗ Could not read frame from camera {cam_index}")
                cap.release()
        else:
            print(f"✗ Could not open camera {cam_index} with backend {backend}")
            
    except Exception as e:
        print(f"✗ Error testing camera {cam_index}: {e}")

print("=== CAMERA TEST COMPLETE ===")

# If everything fails, try reboot suggestion
print("\nIf all tests failed:")
print("1. Check camera cable connection")
print("2. Try: sudo reboot")
print("3. Physical camera hardware may be disconnected")

=== MINIMAL CAMERA TEST ===
Waiting for camera service to stabilize...
Video devices found: ['video0']
Testing direct GStreamer access...
✓ GStreamer can access camera directly
Testing OpenCV camera access...
Trying camera 0 with backend 1800...
✗ Could not open camera 0 with backend 1800
Trying camera 0 with backend 200...
✓ Camera 0 opened with backend 200


[ WARN:0@173.443] global cap_gstreamer.cpp:2784 handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module v4l2src0 reported: Internal data stream error.
[ WARN:0@173.444] global cap_gstreamer.cpp:1679 open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@173.444] global cap_gstreamer.cpp:1164 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@173.444] global cap.cpp:342 open VIDEOIO(GSTREAMER): backend is generally available but can't be used to capture by index


✗ Could not read frame from camera 0


[ WARN:0@183.542] global cap_v4l.cpp:1119 tryIoctl VIDEOIO(V4L2:/dev/video0): select() timeout.


Trying camera 1 with backend 1800...
✗ Could not open camera 1 with backend 1800
=== CAMERA TEST COMPLETE ===

If all tests failed:
1. Check camera cable connection
2. Try: sudo reboot
3. Physical camera hardware may be disconnected


[ WARN:0@184.563] global cap_gstreamer.cpp:2784 handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module v4l2src1 reported: Cannot identify device '/dev/video1'.
[ WARN:0@184.563] global cap_gstreamer.cpp:1679 open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@184.563] global cap_gstreamer.cpp:1164 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@184.563] global cap.cpp:342 open VIDEOIO(GSTREAMER): backend is generally available but can't be used to capture by index
